In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dlib
from imutils import face_utils
import imutils
import cv2
import easydict
import keyboard

In [ ]:
cv2.Vide

In [2]:
def rect_to_bb(rect):
    x=rect.left()
    y=rect.top()
    w=rect.right()-x
    h=rect.bottom()-y
    
    return (x,y,w,h)

In [3]:
def empty_distance(number):
    shape=np.zeros((20,number))
    return shape

In [4]:
def preprocess_dis(dis_array):
    x=dis_array-np.mean(dis_array)
    if np.std(dis_array)!=0:
        x/=np.std(dis_array)
    return x

In [5]:
def convert_points_to_distance(points):

    mid_x=(points[14,0]+points[18,0])/2
    mid_y=(points[14,1]+points[18,1])/2
    mid_point=np.array([mid_x,mid_y])
    points=(points-mid_point)**2
    distances_array=np.sqrt((points[:,0]+points[:,1]))
    
    return preprocess_dis(distances_array)
    

In [6]:
def face_landmarks(image):
    #all_images=[]
    #for image in images:
    gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    rects=detector(gray,1)
    shape=np.zeros((20,2))
    for rect in rects:
        shape=predictor(gray,rect)
        shape=shape_to_np(shape)
        shape=shape[48:68]
    #all_images.append(shape)

    return shape   

In [7]:
def shape_to_np(shape,dtype='int'):
    coords=np.zeros((68,2),dtype=dtype)
    for i in range(0,68):
        coords[i]=(shape.part(i).x,shape.part(i).y)
        
    return coords

In [8]:
args={
    "shape_predictor":"shape_predictor_68_face_landmarks.dat",
    "image":"img3.jpg"
}

In [9]:
detector=dlib.get_frontal_face_detector()
predictor=dlib.shape_predictor(args["shape_predictor"])

In [10]:
from keras.models import load_model
model2=load_model('minor2.h5')

C:\Users\deepu\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [12]:
cap=cv2.VideoCapture(0)
star=False
images_array=[]
while not keyboard.is_pressed('q'):
    ret,frame=cap.read()
    #print(ret)
    if keyboard.is_pressed('s'):
        star=not star
        if star==True:
            images_array=[]
        else:
            images_size=len(images_array)
            """if images_size<=10:
                images_array=np.append(images_array,empty_distance(10-images_size)).reshape(1,10,20)
            else:
                move=images_size/10.0
                images_array=np.array(images_array)
                images_array=images_array[[int(i) for i in np.arange(0,images_size,move)]]
                """          
            images_array=np.array(images_array)
            print(model2.predict_classes(images_array.reshape(1,-1,20)))
        cv2.waitKey(100)
    frame=imutils.resize(frame,width=500)
    frame=cv2.imread("color_001.jpg")
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    rects=detector(gray,1)
    for rect in rects:
        shape=predictor(gray,rect)
        shape=shape_to_np(shape)

        (x,y,w,h)=rect_to_bb(rect)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
        for x,y in shape[0:68]:#[50,51,52,53,54]]:
            cv2.circle(frame,(x,y),1,(0,0,255),-1)

    if star:
        points=face_landmarks(frame)
        points=convert_points_to_distance(points)
        cv2.putText(frame,"Start",(150,150),cv2.FONT_HERSHEY_COMPLEX,1,(150,150,150),2,cv2.LINE_AA)
        images_array.append(points)
    cv2.namedWindow("output",cv2.WINDOW_FULLSCREEN)
    #cv2.setWindowProperty("output",cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_FULLSCREEN)
    cv2.imshow("output",frame)
    
    cv2.waitKey(1)
    
cv2.destroyAllWindows()
cap.release()


In [8]:
cv2.destroyAllWindows()
cap.release()